In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install azure-ai-textanalytics

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

def classify_text(query):
    try:

        ai_endpoint = 'https://sentimentanalysis10.cognitiveservices.azure.com/'
        ai_key = '4kyIh8KGdZYB9j9Yj71gT09yOE3x46rXQpfXilONXKm8CFL7ydK6JQQJ99AJACYeBjFXJ3w3AAAaACOGjCS5'
        project_name = 'MentalHealth10'
        deployment_name = 'MentalHealth'

        # Create client using endpoint and key
        credential = AzureKeyCredential(ai_key)
        ai_client = TextAnalyticsClient(endpoint=ai_endpoint, credential=credential)

        # Prepare the query for classification
        batchedDocuments = [query]

        # Get Classification
        operation = ai_client.begin_single_label_classify(
            batchedDocuments,
            project_name=project_name,
            deployment_name=deployment_name
        )

        document_results = operation.result()

        # Extract classification result
        for classification_result in document_results:
            if classification_result.kind == "CustomDocumentClassification":
                classification = classification_result.classifications[0]
                return classification.category, classification.confidence_score
            elif classification_result.is_error:
                return None, classification_result.error.message

    except Exception as ex:
        return None, str(ex)

In [3]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

def generation(question,model,tokenizer):

    tokenizer = get_chat_template(
        tokenizer,
        chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    )

    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

    category, confidence_score = classify_text(question)

    context = f"انت معالج بالذكاء الاصطناعي قيد التدريب ومهمتك هي تقديم دعم عاطفي مدروس ومخصص لكل مستخدم بناء علي حالته النفسيه الحاليه ستقوم بالاستماع بعنايه الي مخاوفهم ومشاعرهم مع مراعاه ان الحاله النفسيه للمريض هي {category} قم بتقديم استجابات ملاءمه للوضع الذي يمر به استخدم معرفتك بمختلف المناهج العلاجيه لتقديم تقنيات ومحادثات داعمه بلهجه ودوده ومتفهمه تذكر انك مورد للدعم العاطفي والارشاد ولست بديلا عن المعالج البشري تعامل مع المريض بصدق واحترام وكن مرنا في محادثاتك لتتناسب مع حالته النفسيه وحاول التخفيف عنه بطريقه لطيفه ومتوازنه"

    # Create the messages list with the context and user input
    messages = [
        {"from": "system", "value": context},
        {"from": "human", "value": question},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 300, use_cache = True)
    model_answer = tokenizer.batch_decode(outputs)

    return model_answer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained("mohamed517/Arabic-fine-Tuning-LLaMA-Model")

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
question = "اريد ان اشعر بالسعادة"

In [14]:
model_answer = generation(question,model,tokenizer)

In [15]:
import re

# Assuming model_answer is a list with the model's output string at the first index
model_answer_str = model_answer[0]

# Adjusted regex pattern to capture all relevant sections
pattern = re.compile(r'<\|start_header_id\|>(.*?)<\|end_header_id\|>(.*?)<\|eot_id\|>', re.DOTALL)

# Find all matches
sections = pattern.findall(model_answer_str)

# Print each section with clear labels
for role, content in sections:
    role = role.strip().lower()  # Normalize the role for comparison
    content = content.strip()  # Clean up the content
    if role == 'system':
        continue  # Skip the system role
    elif role == 'user':
        print(f"User:\n\n{content}\n")
    elif role == 'assistant':
        print(f"Assistant:\n\n{content}\n")

# Additionally, check for the assistant's response directly after user question if not captured above
if 'assistant' not in [role.strip().lower() for role, _ in sections]:
    assistant_start_index = model_answer_str.find('<|start_header_id|>assistant<|end_header_id|>')
    if assistant_start_index != -1:
        assistant_content = model_answer_str[assistant_start_index:].split('<|eot_id|>')[0]
        print(f"Model Answer:\n\n{assistant_content.replace('<|start_header_id|>', '').replace('<|end_header_id|>', '').strip()}\n")


User:

اريد ان اشعر بالسعادة

Assistant:

تعالج الاسباب النفسيه التي تءدي الي الاكتءاب

